In [1]:
import keras
import tensorflow
import h5py
import numpy as np
import chowdhury_s_model_builder
import os

import tensorflow as tf
from keras.api.models import Sequential, Model, load_model
from keras.api.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, Input, GlobalAveragePooling2D
from keras.api.optimizers import Adam
from keras import regularizers

import tensorflow as tf
print(tf.__version__)

from sklearn.metrics import classification_report, multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

2025-04-26 14:58:06.781863: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-26 14:58:06.794768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745650686.806008   14408 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745650686.809519   14408 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745650686.818738   14408 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

2.19.0


In [2]:
with h5py.File("Data/Mid-Level_Perceptual_Features_Data.h5", "r") as hf:
    mid_level_train_data = hf["train"][:]  # Load the train dataset
    mid_level_label_data = hf["label"][:]  # Load the label dataset

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

(5000, 256, 1292) float32
(5000, 7) int8


In [ ]:
mid_level_train_data = mid_level_train_data[..., np.newaxis]

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

In [ ]:
mid_level_input_shape = mid_level_train_data.shape[1], mid_level_train_data.shape[2], mid_level_train_data.shape[3]
print(mid_level_input_shape)

In [ ]:
model = load_model("fine-tuned_model.keras")
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
x_train_split, x_val, y_train_split, y_val = train_test_split(
    mid_level_train_data, mid_level_label_data, test_size=0.5, random_state=42
)

In [ ]:
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [ ]:
results = model.evaluate(x_val, y_val, batch_size=8, verbose=1)
print("Validation Loss:", results[0])
print("Validation Accuracy:", results[1])
# Predict the labels for the validation set

In [ ]:
x_val.shape, y_val.shape

In [ ]:
single_sample = x_val[2500:]

# Step 1: Predict the probabilities for each label
predictions = model.predict(single_sample)
print("Predictions (Probabilities):", predictions)

# Step 2: Apply the threshold (e.g., 0.4) to each label and convert to binary
threshold = 0.4
predicted_labels = (predictions > threshold).astype(int)
print(f"Predicted labels with threshold {threshold}:", predicted_labels)

# Step 3: For multi-label, you don't need np.argmax, just evaluate the predicted labels directly.
# Here predicted_labels will be a 2D array with shape (1, num_labels), where each entry is 0 or 1.
print("Predicted labels for multi-label classification:", predicted_labels)

# Now, to evaluate the true labels (ground truth) for the same sample
true_labels = y_val[111:112]  # This should also be a binary vector like the predicted labels
print("True labels:", true_labels)


In [ ]:
# Assuming 'model' is your trained model
y_pred = model.predict(x_val)

# Convert probabilities to binary labels (e.g., threshold at 0.5)
y_pred_binary = (y_pred > 0.5).astype(int)

In [ ]:

label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']
# Generate the classification report
report = classification_report(y_val, y_pred_binary, target_names=label_names)
print(report)

# Generate the multilabel confusion matrix
mcm = multilabel_confusion_matrix(y_val, y_pred_binary)
print("Multilabel Confusion Matrix:")
print(mcm)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

# Plot the confusion matrix for each label
for i, cm in enumerate(mcm):
    print(f'Confusion Matrix for {label_names[i]}')
    print('Actual', cm)
    print("Predicted")
    # plt.figure(figsize=(6, 4))
    # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
    # plt.title(f'Confusion Matrix for {label_names[i]}')
    # plt.xlabel('Predicted')
    # plt.ylabel('Actual')
    # plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# import ace_tools as tools
# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

metrics = []

for i, cm in enumerate(mcm):
    tn, fp = cm[0]
    fn, tp = cm[1]
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / cm.sum()
    metrics.append({
        'label': label_names[i],
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy
    })

df_metrics = pd.DataFrame(metrics)
# Display metrics table
print(df_metrics)
# tools.display_dataframe_to_user(name="Label-wise Metrics", dataframe=df_metrics)

# # Plot all confusion matrices in a grid
# fig, axes = plt.subplots(2, 3, figsize=(15, 10))
# axes = axes.flatten()
# for i, cm in enumerate(cms):
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#                 xticklabels=['Pred Neg', 'Pred Pos'],
#                 yticklabels=['True Neg', 'True Pos'],
#                 ax=axes[i])
#     axes[i].set_title(label_names[i])
#     axes[i].set_xlabel('Predicted')
#     axes[i].set_ylabel('Actual')

# plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

thresholds = [0.3, 0.4, 0.5, 0.6]
for t in thresholds:
    preds = (model.predict(x_val) > t).astype(int)
    precision = precision_score(y_val, preds, average='macro')
    recall = recall_score(y_val, preds, average='macro')
    f1 = f1_score(y_val, preds, average='macro')
    accuracy = accuracy_score(y_val, preds)
    print(f"\nThreshold: {t}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"  Accuracy:  {accuracy:.4f}")
